In [111]:
import graphviz
from graphviz import Digraph 

In [6]:
dot = Digraph(comment='The Round Table')

In [7]:
dot

In [8]:
import graphviz

g = graphviz.Digraph('G', filename='hello.gv')

g.edge('Hello', 'World')

g.view()

'hello.gv.pdf'

## Import `dependencyGraph.dot` file and draw a graph

Reference:
- [Gallery of graphviz with example code](https://graphviz.readthedocs.io/en/stable/examples.html) 
- [dot Layout Engine](https://www.graphviz.org/docs/layouts/dot/) 


In [9]:
import pathlib

In [10]:
filepath = pathlib.Path('dependencies-original.dot')

In [11]:
graphviz.render('dot', 'png', filepath).replace('\\', '/')

'dependencies-original.dot.png'

In [ ]:
Import as Digraph

In [112]:
dot = graphviz.Source.from_file('dependencies-original.dot')

list all the nodes with [pygraphviz.AGraph.nodes](https://pygraphviz.github.io/documentation/pygraphviz-1.3rc1/reference/agraph.html#pygraphviz.AGraph.nodes_iter) 


In [113]:
from pygraphviz import *
dot = AGraph("dependencies-original.dot")  

In [5]:
dot.nodes()

['com.elsevier.recs:recs-data-access-spark-lib_2.11:0.1.115-SNAPSHOT',
 'com.fasterxml.jackson.module:jackson-module-scala_2.11:2.11.0',
 'com.fasterxml.jackson.module:jackson-module-paranamer:2.11.0',
 'com.fasterxml.jackson.core:jackson-databind:2.11.0',
 'com.fasterxml.jackson.core:jackson-annotations:2.11.0',
 'com.fasterxml.jackson.core:jackson-core:2.11.0',
 'ch.qos.logback:logback-classic:1.1.2',
 'ch.qos.logback:logback-core:1.1.2',
 'com.typesafe.scala-logging:scala-logging_2.11:3.9.2',
 'org.apache.httpcomponents:fluent-hc:4.5.9',
 'org.apache.httpcomponents:httpclient:4.5.9',
 'commons-codec:commons-codec:1.11',
 'org.apache.httpcomponents:httpcore:4.4.11',
 'commons-logging:commons-logging:1.2',
 'com.newrelic.telemetry:telemetry-core:0.12.0',
 'org.slf4j:slf4j-api:1.7.30',
 'org.slf4j:slf4j-api:1.7.6',
 'org.slf4j:slf4j-api:1.7.26',
 'com.newrelic.telemetry:dropwizard-metrics-newrelic:0.7.0',
 'com.newrelic.telemetry:telemetry-http-okhttp:0.13.2',
 'com.squareup.okhttp3:ok

In [48]:
# extract out libraries without versions
extractCompanyName = lambda x: x.split(':')[0]
extractAppName = lambda x: x.split(':')[1]
extractVersion = lambda x: x.split(':')[2]
extractFullAppName = lambda x: ":".join([extractCompanyName(x), extractAppName(x)])

pairs = list(map(lambda x: (extractFullAppName(x), extractVersion(x)), dot.nodes()))

pairs[:5]

[('com.elsevier.recs:recs-data-access-spark-lib_2.11', '0.1.115-SNAPSHOT'),
 ('com.fasterxml.jackson.module:jackson-module-scala_2.11', '2.11.0'),
 ('com.fasterxml.jackson.module:jackson-module-paranamer', '2.11.0'),
 ('com.fasterxml.jackson.core:jackson-databind', '2.11.0'),
 ('com.fasterxml.jackson.core:jackson-annotations', '2.11.0')]

In [49]:
test_pairs = [('app1', 'v1'), 
              ('app2', 'v1'),
              ('app2', 'v2')
             ]

In [50]:
import itertools
def groupByFullAppName(pairs):
    return { k:list(map(lambda x: x[-1], list(v))) for k, v in itertools.groupby(pairs, lambda x: x[0])}
groupByFullAppName(test_pairs)

{'app1': ['v1'], 'app2': ['v1', 'v2']}

In [51]:
groupByFullAppName(pairs)
# Drop items with no conflicts
appVersionsMapping = groupByFullAppName(pairs)
appVersionsMapping

{'com.elsevier.recs:recs-data-access-spark-lib_2.11': ['0.1.115-SNAPSHOT'],
 'com.fasterxml.jackson.module:jackson-module-scala_2.11': ['2.11.0'],
 'com.fasterxml.jackson.module:jackson-module-paranamer': ['2.11.0'],
 'com.fasterxml.jackson.core:jackson-databind': ['2.11.0'],
 'com.fasterxml.jackson.core:jackson-annotations': ['2.11.0'],
 'com.fasterxml.jackson.core:jackson-core': ['2.11.0'],
 'ch.qos.logback:logback-classic': ['1.1.2'],
 'ch.qos.logback:logback-core': ['1.1.2'],
 'com.typesafe.scala-logging:scala-logging_2.11': ['3.9.2'],
 'org.apache.httpcomponents:fluent-hc': ['4.5.9'],
 'org.apache.httpcomponents:httpclient': ['4.5.9'],
 'commons-codec:commons-codec': ['1.11'],
 'org.apache.httpcomponents:httpcore': ['4.4.11'],
 'commons-logging:commons-logging': ['1.2'],
 'com.newrelic.telemetry:telemetry-core': ['0.12.0'],
 'org.slf4j:slf4j-api': ['1.7.30', '1.7.6', '1.7.26'],
 'com.newrelic.telemetry:dropwizard-metrics-newrelic': ['0.7.0'],
 'com.newrelic.telemetry:telemetry-htt

In [54]:
def filter_no_conflicts(app_versions_map: dict) -> dict:
    return dict(filter(lambda item: len(item[1]) > 1, appVersionsMapping.items()))
appVersionsMapping = filter_no_conflicts(appVersionsMapping)

appVersionsMapping

{'org.slf4j:slf4j-api': ['1.7.30', '1.7.6', '1.7.26'],
 'org.jetbrains.kotlin:kotlin-stdlib': ['1.3.72', '1.3.70'],
 'org.jetbrains.kotlin:kotlin-stdlib-common': ['1.3.72', '1.3.70'],
 'org.checkerframework:checker-qual': ['3.19.0', '3.12.0'],
 'com.google.errorprone:error_prone_annotations': ['2.11.0', '2.10.0']}

Here we have 5 clashed libraries, which means the tool will spit out 5 graphs for each.

Let's take `org.slf4j:slf4j-api` as an example, and draw a Digraph for it.

In [56]:
test_app_versions_map = {'org.slf4j:slf4j-api': ['1.7.30', '1.7.6', '1.7.26']}

In [67]:
def resume_conflict_nodes(conflict_app_name): 
    versions = test_app_versions_map[conflict_app_name]
    conflict_nodes = list(map(lambda version: ":".join([conflict_app_name, version]), versions))
    return conflict_nodes

In [68]:
conflict_app_name = 'org.slf4j:slf4j-api'
conflict_nodes = resume_conflict_nodes(conflict_app_name)
conflict_nodes

['org.slf4j:slf4j-api:1.7.30',
 'org.slf4j:slf4j-api:1.7.6',
 'org.slf4j:slf4j-api:1.7.26']

In [105]:
from typing import Tuple, List

In [102]:
def sort_version_descending(versions: [str]) -> [str]:
    # semver: https://semver.org
    def semver_str_2_ints(version: str) -> Tuple[int]:
        (major, minor, patch) = version.split(".")
        return (int(major), int(minor), int(patch))
    def ints_2_semver(ints: Tuple[int]) -> str:
        (major, minor, patch) = ints
        return ".".join((str(major), str(minor), str(patch)))

    return list(map(ints_2_semver, sorted(list(map(semver_str_2_ints, versions)))))

test_versions = ['1.7.26', '1.7.30', '1.7.6']
expected_versions = ['1.7.6', '1.7.26', '1.7.30']
assert sort_version_descending(test_versions) == expected_versions

To summary:

In [108]:
def find_conflict_nodes(dot):
    extractCompanyName = lambda x: x.split(':')[0]
    extractAppName = lambda x: x.split(':')[1]
    extractVersion = lambda x: x.split(':')[2]
    extractFullAppName = lambda x: ":".join([extractCompanyName(x), extractAppName(x)])

    app_version_pairs = list(map(lambda x: (extractFullAppName(x), extractVersion(x)), dot.nodes()))
    
    def group_by_full_app_name(pairs):
        return { k: (list(map(lambda x: x[-1], list(v)))) for k, v in itertools.groupby(pairs, lambda x: x[0])}
        
    app_versions_map = group_by_full_app_name(app_version_pairs)

    def filter_no_conflicts(app_versions_map: dict) -> dict:
        return dict(filter(lambda item: len(item[1]) > 1, app_versions_map.items()))

    app_versions_map = filter_no_conflicts(app_versions_map)
    
    def sort_version_descending(versions: [str]) -> [str]:
        # semver: https://semver.org
        def semver_str_2_ints(version: str) -> Tuple[int]:
            (major, minor, patch) = version.split(".")
            return (int(major), int(minor), int(patch))
        def ints_2_semver_str(ints: Tuple[int]) -> str:
            (major, minor, patch) = ints
            return ".".join((str(major), str(minor), str(patch)))
        return list(map(ints_2_semver, sorted(list(map(semver_str_2_ints, versions)))))
    
    app_versions_map = { k: sort_version_descending(v) for k, v in app_versions_map.items() }

    def versions_2_nodes(app_name: str, versions: List[str]) -> List[str]: 
        nodes = list(map(lambda ver: ":".join([app_name, ver]), versions))
        return nodes
    
    app_versions_map = { k: versions_2_nodes(k, v) for k, v in app_versions_map.items() }

    return app_versions_map

find_conflict_nodes(dot)

{'org.slf4j:slf4j-api': ['org.slf4j:slf4j-api:1.7.6',
  'org.slf4j:slf4j-api:1.7.26',
  'org.slf4j:slf4j-api:1.7.30'],
 'org.jetbrains.kotlin:kotlin-stdlib': ['org.jetbrains.kotlin:kotlin-stdlib:1.3.70',
  'org.jetbrains.kotlin:kotlin-stdlib:1.3.72'],
 'org.jetbrains.kotlin:kotlin-stdlib-common': ['org.jetbrains.kotlin:kotlin-stdlib-common:1.3.70',
  'org.jetbrains.kotlin:kotlin-stdlib-common:1.3.72'],
 'org.checkerframework:checker-qual': ['org.checkerframework:checker-qual:3.12.0',
  'org.checkerframework:checker-qual:3.19.0'],
 'com.google.errorprone:error_prone_annotations': ['com.google.errorprone:error_prone_annotations:2.10.0',
  'com.google.errorprone:error_prone_annotations:2.11.0']}

In [117]:
canvas = AGraph(strict=True, directed=True)
def add_predecessors_of(node: str, from_dot: AGraph, to_dot: AGraph) -> None:
    precessors = from_dot.predecessors(node)
    for precessor in precessors:
        to_dot.add_node(precessor)
        to_dot.add_edge(precessor, node)
    
    for precessor in precessors:
        add_predecessors_of(precessor, from_dot, to_dot)
    
test_node = 'org.slf4j:slf4j-api:1.7.6'
add_predecessors_of(test_node, dot, canvas)
canvas.nodes()

['ch.qos.logback:logback-classic:1.1.2',
 'org.slf4j:slf4j-api:1.7.6',
 'com.elsevier.recs:recs-data-access-spark-lib_2.11:0.1.115-SNAPSHOT']

In [124]:
canvas = AGraph(strict=True, directed=True)
def add_successors_of(node: str, from_dot: AGraph, to_dot: AGraph) -> None:
    successors = from_dot.successors(node)
    for successor in successors:
        to_dot.add_node(successor)
        
        to_dot.add_edge(node, successor)
    
    for successor in successors:
        add_successors_of(successor, from_dot, to_dot)

test_node = 'org.slf4j:slf4j-api:1.7.6'
add_successors_of(test_node, dot, canvas)
canvas.nodes()

['org.slf4j:slf4j-api:1.7.30', 'org.slf4j:slf4j-api:1.7.6']

Previous outputs `['org.slf4j:slf4j-api:1.7.30', 'org.slf4j:slf4j-api:1.7.6']`, which is odd because 
`org.slf4j:slf4j-api:1.7.6` does NOT depend on `org.slf4j:slf4j-api:1.7.30`.

Looking back into DOT file

```txt
"org.slf4j:slf4j-api:1.7.6" -> "org.slf4j:slf4j-api:1.7.30" [label="Evicted By" style="stroke-dasharray: 5,5"]
```

Because we set exclusive rules (`excludeAll` keyword in sbt file). 

The latter one is we settled on

In [125]:
dot.get_edge('org.slf4j:slf4j-api:1.7.6', 'org.slf4j:slf4j-api:1.7.30')
# 'org.slf4j:slf4j-api:1.7.30',
#  'org.slf4j:slf4j-api:1.7.6',
#  'org.slf4j:slf4j-api:1.7.26',

('org.slf4j:slf4j-api:1.7.6', 'org.slf4j:slf4j-api:1.7.30')

In [ ]:
canvas.layout(prog='dot')
canvas.draw(path="canvas.png")

In [134]:
test_node = 'org.slf4j:slf4j-api:1.7.6'

# dot is global
# 
def draw_graph_for_node(node: str, filename=None, format='png'):
    """Generate a dependency graph with all related dependencies of the node

    Arguments:
    node -- name of conflicted library that is expected to be center of the graph
    filename -- output filename without file extension
    """

    def add_predecessors_of(node: str, from_dot: AGraph, to_dot: AGraph) -> None:
        precessors = from_dot.predecessors(node)
        for precessor in precessors:
            to_dot.add_node(precessor)
            to_dot.add_edge(precessor, node)
        
        for precessor in precessors:
            add_predecessors_of(precessor, from_dot, to_dot)

    def add_successors_of(node: str, from_dot: AGraph, to_dot: AGraph) -> None:
        successors = from_dot.successors(node)
        for successor in successors:
            to_dot.add_node(successor)
            
            to_dot.add_edge(node, successor)
        
        for successor in successors:
            add_successors_of(successor, from_dot, to_dot)

    canvas = AGraph(strict=True, directed=True)
    # populate canvas
    add_predecessors_of(node, dot, canvas)
    add_successors_of(node, dot, canvas)
    # output graph
    canvas.layout(prog='dot')  
    filename = filename or ".".join([node, format])
    canvas.draw(path=filename)

In [ ]:
draw_graph_for_node(node, filename

## Extract all nodes from `DependencyGraph.dot` file

## Find candidates of clashed dependencies

## Prioritise clashed dependencies (manually?)

## For each dependency, assign them same rank and align the related items in ranks